**CONTENT**

In this notebook we will create stochastic simulations based on either `COVID19_SEIRD_sto` or `COVID19_SEIRD_sto_spatial` with various *initial conditions* and *imposed measures*.

**OPEN TASKS**

*short term*
1. ~~Define some interesting realistic and/or artificial initial conditions~~
2. Define some interesting realistic and/or artificial imposed measures: See Tijs' preprint
3. ~~Initiate the models and create a readable environment to make nice plots (create some definitions?)~~
4. ~~Compare the dynamics and results of the spatially stratified model with the national model (these should largely correspond with each other)~~
5. Choose the most pertinent conditions and measures for plotting in geopandas and make a full-resolution video of the simulation
6. ~~Plot infections, hospitalisations and deaths~~
7. *Important*: Compare with actual data (See `MR-time-series-correlation` in `/spatial_private`)
8. ~~Automate the hard-code in `models.py` line `coordinates = [read_coordinates_nis(spatial='test')]`~~
9. ~~Find a nice way to show upper and lower percentiles~~
10. ~~Define functions in a `visualization` environment. That should make it much more readable~~ in `output`
11. ~~Make a Pull Request~~ Make another one
12. **Calibrate the results on the actual data. IDEA: can't we also calibrate on the effect of measures --> see how measures were *actually* followed? Right now this seems like a big estimation.**
13. ~~In `show_map`, include meaning of uncertainty band in legend if only one line is shown~~
14. Define `social_policy_func` in `utils.py`
15. ~~Find a way to save `xarray` objects~~
16. Add social parameters `l` and `tau` to the default fetched parameters
17. ~~Make the plotting device at the end of the Notebook into a function~~
18. Find a way to also easily plot Sciensano data with the same functions
19. Include relative (per 100k) numbers

*long term*  
2. Take out the most relevant results and summarise in a $\LaTeX$ presentation
3. Include age-stratification for the mobility matrix $P^{gh}$
4. **Include spatial stratification for the mobility parameter $p_i$: simulate negative travel advice**
5. Include age stratification for *all* hospital parameters (including average number of days in the hospital)
6. Visualise the measures
7. Include framework for vaccination
8. Allow for fractional infection from mildly symptomatic people that ignore safety regulations (self-isolation), preferably based on research
9. Show age distribution in the relevant time series at any time: add this to function.
10. Fix the percentile problem: right now we are taking the percentiles for every time value, but independent simulations might cross each other. This is therefore not strictly correct (this is a detail though)
11. In the `show_map` function, include age stratification.
12. Add 'smooth-out' option (7-day moving average) to show_maps and show_graphs functions, as is done in the actual data

**OPEN QUESTIONS**

1. What is the reason for and the meaning of the normalisation factor $z_i$ in `COVID19_SEIRD_sto_spatial`?
2. Is it OK for all susceptibility factors $s_i$ to be smaller than one, as is the case now? Not a big deal, but seems odd and certainly changes the $\beta$ value (such that it doesn't correspond to that of other research groups)
3. Can we simulate home isolation within this framework? If the contact matrix $N_{c,ij}$ is set to only $N_{home,ij}$, this still assumes homogeneity within each patch. One than still has contact with the entire patch, which is not realistic
4. Only professional commuting is included, but presumably this is not sufficient. Can we use these simulations to verify that? **MEETING 29 OCTOBER**
5. ~~Why are there so few hospitalisations compared to the number of deaths? There is definitely a bug there somewhere: the number of ICUs is even larger than the number of hospitalisations, which cannot be correct.~~
    * Probably because `H_in` literally shows how many new people get in at any day (flux only), while for example `ICU` shows how many people are *actively* occupying ICU beds. Of course there is a crucial difference between the two.
6. ~~Why does it take SOOO much longer to compute the national model vs the spatial model? Verify whether this is a factor $G$ difference~~ Reason: the figure environment should be reloaded every time, otherwise the figures are overwritten, which takes a lot of memory.
7. All numbers go up in the aggregated version compared to the national model. I would actually expect the opposite, because people aren't spread as homogenic as before. What is going on? This must be **the effect of the normalisation** ...but wouldn't this give the opposite effect? $z_i \times f$ is of the order $0.5 \times 1.5$ which brings *down* the infectivity, which lowers the probability of being infected in a patch $g$. So if anything, I would expect the time series to be smaller than on the national level.
8. How do I implement multiple policy changes?

# Load packages

In [ ]:
# Established packages
import os
import numpy as np
import pandas as pd
import geopandas as gp
import datetime
import math
import xarray as xr # labels in the form of dimensions, coordinates and attributes
import matplotlib.pyplot as plt
import zarr

# Custom package covid19model
from covid19model.models import models
from covid19model.models.utils import name2nis, social_policy_func, save_sim, open_sim
from covid19model.data import model_parameters #, sciensano, google
from covid19model.visualization.output import population_status, infected, show_map, show_graphs


# Load different geographical aggregations
maps_dir = '../data/raw/geopandas/BE/'

country = gp.read_file(os.path.join(maps_dir, 'AD_6_Country.shp'))
regions = gp.read_file(os.path.join(maps_dir, 'AD_5_Region.shp'))
provinces = gp.read_file(os.path.join(maps_dir, 'AD_4_Province.shp'))
arrondissements = gp.read_file(os.path.join(maps_dir, 'AD_3_District.shp'))
municipalities = gp.read_file(os.path.join(maps_dir, 'AD_2_Municipality.shp'))
testgeo = arrondissements.loc[arrondissements['NISCode'].isin(['11000', '21000', '44000'])].reset_index(drop=True)
# Add NIS-code to Arrondissement Brussel-Hoofdstad
provinces.loc[provinces['NISCode']=='NA', 'NISCode'] = '21000'


# OPTIONAL: Load the "autoreload" extension so that package code can change
%load_ext autoreload
# OPTIONAL: always reload modules so that as you change code in src, it gets loaded
# This may be useful because the `covid19model` package is under construction
%autoreload 2

# Content and initialisation

## Oversight of various model conditions

Any of the conditions below are implemented for both the spatial and the national stochastic SEIRD-extended model.

**Initial conditions**
1. One exposed person on day 0. With some spatial variation:  
    1. On the far southeast side of the country  
    2. On the far northwest side of the country  
    3. In Brussels  
2. Fifty exposed people on day 0, same spatial variation
3. Only asymptomatic people instead of exposed people
4. Only presymptomatic people instead of exposed people
5. No mobility whatsoever (sanity check, can only be done in spatial model)

**Measures**
1. Decrease overall contact parameter: type of contact stays the same, but the frequency is lowered
2. Change *type* of contact: only remain home contact (but note that this does not mean that families are isolated!)
3. Decrease mobility between cities (can only be done in spatial model)
4. Vaccination: migrate the number of Susceptible straight to the Recovered category

**Spatial levels**
1. 'Test' level: only arrondissements Antwerpen, Brussel, Gent
2. Arrondissement level: 43 arrondissements
3. Province level: 10+1 provinces
4. National level: no spatial stratification

## Initialisation of common parameters

In [ ]:
# ---------------------------- #
# Choose simulation parameters #
# ---------------------------- #

# Note that a high number of simulations is computationally expensive, especially for high spatial stratification
sim_number = 100
days_number = 100

# Policy values (for more details on nature of measures, see block below)
policy_time = 40 # Must be smaller than days_number
tau = 5 # days until new measures show some effect
l = 5 # days since policy_time + tau until new measures show full effect

# Sanity check: correspondence between spatial model and national model

Below we load *a single exposed 35 year old individual* in every arrondissement (43 exposed people). We compare the resulting time series with those from simulations from the model stratified per province (4 per province+Bxl), and from the national model (43 in Belgium). Ideally, these should show identical dynamics.

**Results**
1. The simulations on province level and on arrondissement level are virtually identical.
2. The national model predicts gives many *fewer* hospitalisations.

**Discussion**
1. It is still unclear what the prefactors $f_i$ and $z_i$ mean. These influence the effective $\beta$, hence influencing the speed of the infection.
2. Currently $\beta$ is effectively *smaller* for the spatial model, so I don't understand why the infection spreads faster than in the national model
3. Perhaps I should spread the infected people out more according to population density (so more in Brussels and fewer in Arlon)

**Save simulation results in zarr**

In [ ]:
# Name of the zarr directory in which the simulations are to be saved
zarr_dir = '../data/simulated/'
zarr_name = "Sanity-check_" + str(sim_number) + "sims_" + str(days_number) + \
                    "days_" + "Nctot-to-Nchome-day" + str(policy_time) + '.zarr'

zarr = zarr_dir + '/' + zarr_name

## Spatial model on arrondissement level

### Prepare the simulation

In [ ]:
# Name of the zarr gropu in which this particular simulation is or will be saved
group = "arr_1E-per-arr"

# Load parameters and interaction matrices
age_str = True
spatial_str = 'arr'
intensity='all'
params = model_parameters.get_COVID19_SEIRD_parameters(age_stratified=age_str, spatial=spatial_str)
initN, Nc_home, Nc_work, Nc_schools, Nc_transport, Nc_leisure, Nc_others, Nc_total = model_parameters.get_interaction_matrices(intensity=intensity, spatial=spatial_str)

# Add social policy parameter values
params.update({'l': l,
              'tau': tau,
              'policy_time' : policy_time})

In [ ]:
# Define initial conditions

# Every arrondissements gets one 35-year-old exposed individual
exposed = np.zeros(initN.shape)
for arr in exposed:
    arr[3] = 1
init_states = dict({'S' : initN, 'E' : exposed})

# The type of policy change: contact is reduce from everything to home contact (within the arrondissement) only
policy1 = Nc_total
policy2 = Nc_home
params.update({'policy1' : policy1,
               'policy2' : policy2})

In [ ]:
# Load the model
model = models.COVID19_SEIRD_sto_spatial(init_states, params, time_dependent_parameters={'Nc' : social_policy_func},
                                     discrete=True, spatial='arr')

### Run and save the simulations - WARNING: computationally expensive (only do once)

In [ ]:
# Group name and output description
description = f"Stochastic spatial SEIRD extended model with {sim_number} "\
                    f"parallel simulations in 43 arrondissements over {days_number} days."\
                    f"At day 0 a single exposed person in the age class 30-40 is released in"\
                    f"each of the arrondissements. At day {policy_time} measures are imposed, "\
                    f"bringing down the contact rate from Nc_total to Nc_home over the course of "\
                    f"{tau} + {l} days (tau and l compliance parameters) reaches full effect."

# Perform simulation (commented out for safety)
# WARNING! First check if there isn't already data available for this simulation!
# out_sanity_arr = model.sim(days_number, N=sim_number, verbose=True)

# Save simulation results
# save_sim(out_sanity_arr, zarr, group, descr=description)

### Inspect the results

In [ ]:
# Load the simulation data
out_sanity_arr = open_sim(zarr, group, verbose=False)

# This goes wrong! For that reason the creation of the figures goes wrong as well.
out_sanity_arr.quantile(0.5, dim='draws')

In [ ]:
# Show map of a snapshot and the graphs of the major urban hubs (Antwerp, Brussels, Ghent)
day = 60
ts_geo = 'E'
ts_graph = ['E', 'ICU', 'D']
nis = [11000, 21000, 44000]
geomap, graphs = show_map(out_sanity_arr, arrondissements, ts_geo=ts_geo, day=day, ts_graph=ts_graph)

In [ ]:
figname = "sanity_arr_50sim_E-Hin-ICU-D.jpg"
graphs = show_graphs(out_sanity_arr, ylim=[30000, 200, 400, 500])#, figname=figname)

## Spatial model on province level

### Prepare the simulation

In [ ]:
# Name of the zarr gropu in which this particular simulation is or will be saved
group = "prov_4E-per-prov"

# Load parameters and interaction matrices
age_str = True
spatial_str = 'prov'
intensity='all'
params = model_parameters.get_COVID19_SEIRD_parameters(age_stratified=age_str, spatial=spatial_str)
initN, Nc_home, Nc_work, Nc_schools, Nc_transport, Nc_leisure, Nc_others, Nc_total = model_parameters.get_interaction_matrices(intensity=intensity, spatial=spatial_str)

# Add social policy parameter values
params.update({'l': l,
              'tau': tau,
              'policy_time' : policy_time})

In [ ]:
# Define initial conditions

# Every province gets four 35-year-old exposed individuals
exposed = np.zeros(initN.shape)
for prov in exposed:
    prov[3] = 4
init_states = dict({'S' : initN, 'E' : exposed})

# The type of policy change: contact is reduce from everything to home contact (within the arrondissement) only
policy1 = Nc_total
policy2 = Nc_home
params.update({'policy1' : policy1,
               'policy2' : policy2})

In [ ]:
# Load the model
model = models.COVID19_SEIRD_sto_spatial(init_states, params, time_dependent_parameters={'Nc' : social_policy_func},
                                     discrete=True, spatial='prov')

### Run and save the simulations - WARNING: computationally expensive (only do once)

In [ ]:
# Group name and output description
description = f"Stochastic spatial SEIRD extended model with {sim_number} "\
                    f"parallel simulations in 10+1 provinces over {days_number} days. "\
                    f"At day 0 four exposed people in the age class 30-40 are released in "\
                    f"each of the provinces + Brussels-Capital. At day {policy_time} measures are imposed, "\
                    f"bringing down the contact rate from Nc_total to Nc_home. Over the course of "\
                    f"{tau} + {l} days (tau and l parameters) compliance reaches full effect."

# Perform simulation (commented out for safety)
# WARNING! First check if there isn't already data available for this simulation!
# out_sanity_prov = model.sim(days_number, N=sim_number, verbose=True)
out_sanity_test = model.sim(days_number, N=3, verbose=True)

out_sanity_test

# Save simulation results
# save_sim(out_sanity_prov, zarr, group, descr=description)

### Inspect the results

In [ ]:
# Load the simulation data
out_sanity_prov = open_sim(zarr, group)

In [ ]:
# Show map of a snapshot and the graphs of the major urban hubs (Antwerp, Brussels, Ghent)
day = 60
ts_geo = 'E'
ts_graph = ['E', 'ICU', 'D']
nis = [10000, 21000, 40000]
geomap, graphs = show_map(out_sanity_prov, provinces, ts_geo=ts_geo, day=day, ts_graph=ts_graph)

In [ ]:
graphs = show_graphs(out_sanity_prov, ylim=[30000, 200, 400, 500], figname="sanity_prov_50sim_E-Hin-ICU-D.jpg")

## National model

### Prepare the simulation

In [ ]:
# Name of the zarr gropu in which this particular simulation is or will be saved
group = "nat_43E"

# Load parameters and interaction matrices
age_str = True
intensity='all'
# spatial_str has been left out here
params = model_parameters.get_COVID19_SEIRD_parameters(age_stratified=age_str)
initN, Nc_home, Nc_work, Nc_schools, Nc_transport, Nc_leisure, Nc_others, Nc_total = model_parameters.get_interaction_matrices(intensity=intensity)

# Add social policy parameter values
params.update({'l': l,
              'tau': tau,
              'policy_time' : policy_time})

In [ ]:
# Define initial conditions

# The entire country gets 43 35-year-old exposed individuals
exposed = np.zeros(initN.shape)
exposed[3] = 43
init_states = dict({'S' : initN, 'E' : exposed})

# The type of policy change: contact is reduce from everything to home contact (within the arrondissement) only
policy1 = Nc_total
policy2 = Nc_home
params.update({'policy1' : policy1,
               'policy2' : policy2})

In [ ]:
# Load the model
model = models.COVID19_SEIRD_sto(init_states, params, time_dependent_parameters={'Nc' : social_policy_func},
                                     discrete=True)

### Run and save the simulations - WARNING: computationally expensive (only do once)

In [ ]:
# Group name and output description
description = f"Stochastic national SEIRD extended model with {sim_number} "\
                    f"parallel simulations over {days_number} days. "\
                    f"At day 0 43 exposed people in the age class 30-40 are released homogeneously distributed "\
                    f"over the whole of Belgium. At day {policy_time} measures are imposed, "\
                    f"bringing down the contact rate from Nc_total to Nc_home. Over the course of "\
                    f"{tau} + {l} days (tau and l parameters) compliance reaches full effect."

# Perform simulation (commented out for safety)
# WARNING! First check if there isn't already data available for this simulation!
# out_sanity_nat = model.sim(days_number, N=sim_number, verbose=True)

# Save simulation results
# save_sim(out_sanity_nat, zarr, group, descr=description)

### Inspect the results

In [ ]:
# Load the simulation data
out_sanity_nat = open_sim(zarr, group)

In [ ]:
graphs = show_graphs(out_sanity_nat,ylim=[30000, 200, 400, 500])

# Demonstrating the spatial effect

Here the power of the spatial model is shown by dropping 5 infected patients in 'the middle of nowhere' and seeing how their disease spreads through Belgium.

The mean purpose here is to communicate the working of the simulation

**Result**  
Can be seen as an mp4 in the archives (Downloads folder at the moment)

**Discussion**  
1. This looks realistic
2. Would be interesting to treat these results as mock data and perform data analysis on these
3. Arguably more interesting to show the relative impact rather than the absolute impact

## Prepare the simulation

In [ ]:
# Load parameters and interaction matrices
age_str = True
intensity='all'
spatial_str = 'arr'
params = model_parameters.get_COVID19_SEIRD_parameters(age_stratified=age_str, spatial=spatial_str)
initN, Nc_home, Nc_work, Nc_schools, Nc_transport, Nc_leisure, Nc_others, Nc_total = model_parameters.get_interaction_matrices(intensity=intensity, spatial=spatial_str)

# Add social policy parameter values
params.update({'l': l,
              'tau': tau,
              'policy_time' : policy_time})

In [ ]:
# Define initial conditions

# Put 5 35 year-old exposed people in Arlon (index 35)
exposed = np.zeros(initN.shape)
exposed[35,3] = 5
init_states = dict({'S' : initN, 'E' : exposed})

# The type of policy change: contact is reduce from everything to home contact (within the arrondissement) only
policy1 = Nc_total
policy2 = Nc_home
params.update({'policy1' : policy1,
               'policy2' : policy2})

In [ ]:
# Load the model
model = models.COVID19_SEIRD_sto_spatial(init_states, params, time_dependent_parameters={'Nc' : social_policy_func},
                                     discrete=True)

## Run the simulation - Warning: computationally expensive

In [ ]:
# Commented out for safety

out_demo_arr = model.sim(days_number, N=sim_number, verbose=True)

## Inspect the results

In [ ]:
# Make a video: see archives

ts_geo = 'E'
ts_graph = ['E', 'ICU', 'D']
nis = [81000, 21000, 31000] # Arlon, Brussel, Brugge

# Commented out for safety

# for day in range(days_number+1):
#     figname = f"demo_arr_5-35-yo_81000_day{day}.jpg"
#     maps, graphs = show_map(out_demo_arr, arrondissements, ts_geo=ts_geo, day=day, ts_graph=ts_graph, figname=figname)

## Plotting in graph

In [ ]:
z = 4

if z == 4:
    print('yes')
elif z:
    print('also yes')